In [2]:
!pip install torch

     ------------------------------------- 161.9/161.9 MB 12.3 MB/s eta 0:00:00


In [6]:
import torch
import nltk
from functools import reduce
from torch.utils.data import Dataset
from model import GRULanguageModel
from dataset import GRULanguageModelDataset, preprocess
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch import nn

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ziipp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
text = 'she sells sea shells by the sea shore'

In [9]:
dataset = GRULanguageModelDataset(text)
for d in dataset:
    print(d)
    break

tensor([ 1,  4,  5,  6,  7,  8,  9,  6, 10,  2])


d:\Study\NotebookProjects\hello-transformer\chapter1\dataset.py:64: UserWarning: Failed to initialize NumPy: numpy.core.multiarray failed to import (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:68.)
  return torch.tensor(indice)


In [10]:
dataset.vocab

{'<pad>': 0,
 '<s>': 1,
 '</s>': 2,
 '<unk>': 3,
 'she': 4,
 'sells': 5,
 'sea': 6,
 'shells': 7,
 'by': 8,
 'the': 9,
 'shore': 10}

## 데이터로더 만들기

In [11]:
def collate_fn(batch):
    batch = pad_sequence(batch, batch_first=True)
    return batch

In [12]:
dataloader = DataLoader(dataset, collate_fn=collate_fn, batch_size=16)

In [13]:
for d in dataloader:
    print(d)
    print(d.shape)
    break

tensor([[ 1,  4,  5,  6,  7,  8,  9,  6, 10,  2]])
torch.Size([1, 10])


## 학습

In [15]:
!pip install numpy

     --------------------------------------- 14.7/14.7 MB 46.7 MB/s eta 0:00:00


In [17]:
!python run_gru.py --hidden_size=30

0th iteration -> loss=54.9989
0th iteration -> loss=57.3448
0th iteration -> loss=54.6769
1th iteration -> loss=54.0226
1th iteration -> loss=54.3755
1th iteration -> loss=54.3128
2th iteration -> loss=55.4618
2th iteration -> loss=55.1277
2th iteration -> loss=53.7750
3th iteration -> loss=53.0005
3th iteration -> loss=53.8513
3th iteration -> loss=53.9846
4th iteration -> loss=52.7672
4th iteration -> loss=51.7067
4th iteration -> loss=52.8269
5th iteration -> loss=51.4072
5th iteration -> loss=51.1293
5th iteration -> loss=51.1255
6th iteration -> loss=50.0746
6th iteration -> loss=49.3236
6th iteration -> loss=50.2648
7th iteration -> loss=48.4280
7th iteration -> loss=48.0604
7th iteration -> loss=49.3110
8th iteration -> loss=47.2281
8th iteration -> loss=47.0946
8th iteration -> loss=48.6680
9th iteration -> loss=46.7307
9th iteration -> loss=46.5584
9th iteration -> loss=48.2874
10th iteration -> loss=45.6019
10th iteration -> loss=45.4592
10th iteration -> loss=48.1107
11th it

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ziipp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## GRULanguageModel를 이용한 문장 생성하기

In [19]:
import pickle
from generate import generate_sentence_from_bos

In [20]:
# define dataset and dataloader
vocab = pickle.load(open('vocab.pickle', 'rb'))

In [21]:
# define and load model
hidden_size = 30
output_size = len(vocab)
model = GRULanguageModel(hidden_size=hidden_size, output_size=output_size)
model.load_state_dict(torch.load('gru_model.bin'))
model.eval()

GRULanguageModel(
  (embedding): Embedding(21, 30)
  (gru): GRU(30, 30, batch_first=True)
  (softmax): LogSoftmax(dim=-1)
  (out): Linear(in_features=30, out_features=21, bias=True)
)

In [22]:
generated_text = generate_sentence_from_bos(model, vocab, bos=1)

In [23]:
generated_text

"<s> for if she sells sea shells by the sea shore then i'm sure she sells sea shore shells."